In [ ]:
import tensorflow as tf
tf.random.set_seed(1)
tf.compat.v1.enable_eager_execution()

import numpy as np
np.random.seed(1)

import gensim
import pandas as pd
import matplotlib.pyplot as plt
import talos as ta

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, \
    SpatialDropout1D, Bidirectional, Conv1D, concatenate
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import Callback

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix

%matplotlib inline

In [ ]:
print(tf.test.gpu_device_name())

/device:GPU:0


In [ ]:
df = pd.read_csv('../data/data.csv')

In [ ]:
df.head()

,word,tag
0,Pengamat,O
1,politik,O
2,dari,O
3,Universitas,B-ORGANIZATION
4,Gadjah,I-ORGANIZATION


In [ ]:
sentences = []
cnt = 1

for i in df.itertuples():
    sentences.append(cnt)
    
    if '.' in str(i.word):
        cnt += 1
        
df['sentence #'] = sentences
df.head()

,word,tag,sentence #
0,Pengamat,O,1
1,politik,O,1
2,dari,O,1
3,Universitas,B-ORGANIZATION,1
4,Gadjah,I-ORGANIZATION,1


In [ ]:
agg_func = lambda s: [(w, t) for w, t in zip(s['word'].values.tolist(), s['tag'].values.tolist())]
grouped = df.groupby('sentence #').apply(agg_func)
sentences = [s for s in grouped]

In [ ]:
words = list(set(df['word'].values))
words.append('PADDING')
num_words = len(words)
tags = list(set(df['tag'].values))
num_tags = len(tags)

In [ ]:
tags = sorted([t for t in tags if t != 'O'], key=lambda x: (x[2], x[0]))
tags.append('O')

In [ ]:
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [ ]:
tag2idx

{'B-LOCATION': 0,
 'B-ORGANIZATION': 4,
 'B-PERSON': 8,
 'B-QUANTITY': 12,
 'B-TIME': 16,
 'I-LOCATION': 1,
 'I-ORGANIZATION': 5,
 'I-PERSON': 9,
 'I-QUANTITY': 13,
 'I-TIME': 17,
 'L-LOCATION': 2,
 'L-ORGANIZATION': 6,
 'L-PERSON': 10,
 'L-QUANTITY': 14,
 'L-TIME': 18,
 'O': 20,
 'U-LOCATION': 3,
 'U-ORGANIZATION': 7,
 'U-PERSON': 11,
 'U-QUANTITY': 15,
 'U-TIME': 19}

### Pad Sequence

In [ ]:
max_len = 40
X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding='post', value=num_words-1)

y = [[tag2idx[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding='post', value=tag2idx['O'])
y = [to_categorical(i, num_classes=num_tags) for i in y]

In [ ]:
x_train, x_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.3, random_state=1)
x_val, x_test, y_val, y_test = train_test_split(x_val_test, y_val_test, test_size=0.5, random_state=1)

### Pre-trained Word2Vec Embedding

In [ ]:
model = gensim.models.Word2Vec.load("../checkpoint/w2vec_wiki_id_case")

In [ ]:
embedding_matrix = np.zeros((len(words), 400))

for i, w in enumerate(words):
    try:
        embedding_vector = model.wv[w]
        embedding_matrix[i] = embedding_vector
    except:
        embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25), 400)

Initial parameters

In [ ]:
def f1_score_micro(y_true, y_pred):
    y_true = np.argmax(y_true.numpy(), axis=-1).reshape(-1)
    y_pred = np.argmax(y_pred.numpy(), axis=-1).reshape(-1)
    
    # label O will be ignored during training and evaluation
    o_pad_idx = np.where(y_true==20) # 20 is the idx for label O
    
    # remove label O
    y_true = np.delete(y_true, o_pad_idx)
    y_pred = np.delete(y_pred, o_pad_idx)
    
    # compute f1 score with micro average
    score = f1_score(y_true, y_pred, average='micro')
    
    return score

In [ ]:
def create_model(x_train, y_train, x_val, y_val, params):
    input_sequence = Input(shape=(max_len, ))
    
    model = Embedding(
        input_dim= embedding_matrix.shape[0], 
        weights=[embedding_matrix], 
        output_dim=embedding_matrix.shape[1], 
        input_length=max_len,
        trainable=False
    )(input_sequence)
    
    # dropout layer
    model = SpatialDropout1D(params['dropout'])(model)
            
    # lstm layer
    model = Bidirectional(LSTM(units=params['lstm_units'], return_sequences=True))(model)

    # convolution layer
    model2g = Conv1D(params['filters'], 2, activation='relu', padding='same')(model)
    model4g = Conv1D(params['filters'], 4, activation='relu', padding='same')(model)
    model6g = Conv1D(params['filters'], 6, activation='relu', padding='same')(model)
    model8g = Conv1D(params['filters'], 8, activation='relu', padding='same')(model)
    model10g = Conv1D(params['filters'], 10, activation='relu', padding='same')(model)
    
    model = concatenate([model2g, model4g, model6g, model8g, model10g])
    
    # output layer
    output_sequence = Dense(num_tags, activation='softmax')(model)
    
    # model
    model = Model(input_sequence, output_sequence)
    
    # learning algorithm (optimizer)
    if params['optimizer'] == 'Nadam':
        optm = tf.keras.optimizers.Nadam(lr=params['lr'])
        
    if params['optimizer'] == 'Adam':
        optm = tf.keras.optimizers.Adam(lr=params['lr'])
        
    if params['optimizer'] == 'RMSprop':
        optm = tf.keras.optimizers.RMSprop(lr=params['lr'])
    
    loss = tf.keras.losses.CategoricalCrossentropy()

    model.compile(loss=loss, optimizer=optm, metrics=[f1_score_micro], run_eagerly=True)

    early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_f1_score_micro', mode='max')

    history = model.fit(
      x_train, np.array(y_train),
      validation_data=(x_val, np.array(y_val)),
      epochs=100, verbose=1, callbacks=[early_stopping], batch_size=params['batch_size']
    )

    return history, model

In [ ]:
params = {
    'optimizer': ['Adam'],
    'lr': [0.01],
    'filters': [64],
    'lstm_units': [100],
    'dropout': [0.1, 0.3, 0.5, 0.7],
    'batch_size': [128]
}

In [ ]:
t = ta.Scan(x=x_train,
    y=np.array(y_train),
    x_val=x_val,
    y_val=np.array(y_val),
    model=create_model,
    params=params,
    experiment_name='bilstm_cnns_w2v_opt_v3', 
    val_split=None
)


  0%|          | 0/4 [00:00<?, ?it/s]

Epoch 1/100
27/27 [==============================] - 1s 51ms/step - loss: 0.8491 - f1_score_micro: 0.0204 - val_loss: 0.3834 - val_f1_score_micro: 0.0595
Epoch 2/100
27/27 [==============================] - 1s 48ms/step - loss: 0.3041 - f1_score_micro: 0.1789 - val_loss: 0.2398 - val_f1_score_micro: 0.3062
Epoch 3/100
27/27 [==============================] - 1s 46ms/step - loss: 0.1934 - f1_score_micro: 0.4234 - val_loss: 0.1805 - val_f1_score_micro: 0.4592
Epoch 4/100
27/27 [==============================] - 1s 44ms/step - loss: 0.1391 - f1_score_micro: 0.5888 - val_loss: 0.1494 - val_f1_score_micro: 0.5944
Epoch 5/100
27/27 [==============================] - 1s 45ms/step - loss: 0.1101 - f1_score_micro: 0.6819 - val_loss: 0.1370 - val_f1_score_micro: 0.6671
Epoch 6/100
27/27 [==============================] - 1s 48ms/step - loss: 0.0918 - f1_score_micro: 0.7397 - val_loss: 0.1345 - val_f1_score_micro: 0.6654
Epoch 7/100
27/27 [==============================] - 1s 43ms/step - loss: 0.


 25%|██▌       | 1/4 [01:07<03:22, 67.49s/it]

Epoch 1/100
27/27 [==============================] - 1s 45ms/step - loss: 1.3878 - f1_score_micro: 0.0015 - val_loss: 1.3077 - val_f1_score_micro: 0.0000e+00
Epoch 2/100
27/27 [==============================] - 1s 44ms/step - loss: 1.3375 - f1_score_micro: 0.0015 - val_loss: 1.2841 - val_f1_score_micro: 0.0399
Epoch 3/100
27/27 [==============================] - 1s 44ms/step - loss: 0.9506 - f1_score_micro: 0.0141 - val_loss: 0.4169 - val_f1_score_micro: 0.0328
Epoch 4/100
27/27 [==============================] - 1s 45ms/step - loss: 0.3506 - f1_score_micro: 0.0588 - val_loss: 0.2932 - val_f1_score_micro: 0.1204
Epoch 5/100
27/27 [==============================] - 1s 46ms/step - loss: 0.2504 - f1_score_micro: 0.2517 - val_loss: 0.2207 - val_f1_score_micro: 0.3129
Epoch 6/100
27/27 [==============================] - 1s 44ms/step - loss: 0.1965 - f1_score_micro: 0.3863 - val_loss: 0.1880 - val_f1_score_micro: 0.4007
Epoch 7/100
27/27 [==============================] - 1s 45ms/step - loss


 50%|█████     | 2/4 [01:37<01:52, 56.22s/it]

Epoch 1/100
27/27 [==============================] - 1s 50ms/step - loss: 0.6476 - f1_score_micro: 0.0109 - val_loss: 0.3484 - val_f1_score_micro: 0.0420
Epoch 2/100
27/27 [==============================] - 1s 48ms/step - loss: 0.3086 - f1_score_micro: 0.1095 - val_loss: 0.2478 - val_f1_score_micro: 0.2707
Epoch 3/100
27/27 [==============================] - 1s 47ms/step - loss: 0.2192 - f1_score_micro: 0.3425 - val_loss: 0.1853 - val_f1_score_micro: 0.4390
Epoch 4/100
27/27 [==============================] - 1s 44ms/step - loss: 0.1714 - f1_score_micro: 0.4860 - val_loss: 0.1550 - val_f1_score_micro: 0.5332
Epoch 5/100
27/27 [==============================] - 1s 48ms/step - loss: 0.1472 - f1_score_micro: 0.5651 - val_loss: 0.1418 - val_f1_score_micro: 0.6452
Epoch 6/100
27/27 [==============================] - 1s 49ms/step - loss: 0.1282 - f1_score_micro: 0.6262 - val_loss: 0.1272 - val_f1_score_micro: 0.6513
Epoch 7/100
27/27 [==============================] - 1s 45ms/step - loss: 0.


 75%|███████▌  | 3/4 [02:10<00:49, 49.23s/it]

Epoch 1/100
27/27 [==============================] - 1s 44ms/step - loss: 0.6548 - f1_score_micro: 0.0098 - val_loss: 0.3439 - val_f1_score_micro: 0.0700
Epoch 2/100
27/27 [==============================] - 1s 45ms/step - loss: 0.3064 - f1_score_micro: 0.1125 - val_loss: 0.2442 - val_f1_score_micro: 0.2805
Epoch 3/100
27/27 [==============================] - 1s 46ms/step - loss: 0.2268 - f1_score_micro: 0.3367 - val_loss: 0.1880 - val_f1_score_micro: 0.4296
Epoch 4/100
27/27 [==============================] - 1s 48ms/step - loss: 0.1883 - f1_score_micro: 0.4413 - val_loss: 0.1586 - val_f1_score_micro: 0.4944
Epoch 5/100
27/27 [==============================] - 1s 44ms/step - loss: 0.1661 - f1_score_micro: 0.5040 - val_loss: 0.1468 - val_f1_score_micro: 0.5980
Epoch 6/100
27/27 [==============================] - 1s 43ms/step - loss: 0.1502 - f1_score_micro: 0.5524 - val_loss: 0.1410 - val_f1_score_micro: 0.6792
Epoch 7/100
27/27 [==============================] - 1s 47ms/step - loss: 0.


100%|██████████| 4/4 [02:53<00:00, 43.44s/it]


In [ ]:
# round 4
t.data[['loss', 'f1_score_micro', 'val_loss', 'val_f1_score_micro', 'dropout']] \
    .sort_values(by=['val_f1_score_micro'], ascending=False)

,loss,f1_score_micro,val_loss,val_f1_score_micro,dropout
3,0.048002,0.863403,0.137955,0.763460,0.7
1,0.040812,0.891935,0.135959,0.736122,0.3
2,0.033616,0.907585,0.144163,0.724609,0.5
0,0.006004,0.986169,0.270796,0.702142,0.1


In [ ]:
# round 3
t.data[['loss', 'f1_score_micro', 'val_loss', 'val_f1_score_micro', 'filters']] \
    .sort_values(by=['val_f1_score_micro'], ascending=False)

,loss,f1_score_micro,val_loss,val_f1_score_micro,filters
1,0.021418,0.942006,0.165591,0.732211,64
0,0.016788,0.954940,0.182246,0.724843,32
2,0.034370,0.907809,0.139293,0.712943,128
3,1.342312,0.000000,1.307674,0.000000,256


In [ ]:
# round 2
t.data[['loss', 'f1_score_micro', 'val_loss', 'val_f1_score_micro', 'lstm_units']] \
    .sort_values(by=['val_f1_score_micro'], ascending=False)

,loss,f1_score_micro,val_loss,val_f1_score_micro,lstm_units
1,0.023470,0.936328,0.154423,0.743864,100
0,0.018684,0.951146,0.180076,0.721642,50
2,0.024138,0.935081,0.160149,0.715991,200


In [ ]:
# round 1
t.data[['loss', 'f1_score_micro', 'val_loss', 'val_f1_score_micro', 'optimizer', 'lr']] \
    .sort_values(by=['val_f1_score_micro'], ascending=False)

,loss,f1_score_micro,val_loss,val_f1_score_micro,optimizer,lr
1,0.033131,0.909018,0.155255,0.749481,Adam,0.0100
2,0.018617,0.959671,0.242784,0.725582,RMSprop,0.0100
0,0.014270,0.961801,0.200763,0.722709,Nadam,0.0100
4,0.053579,0.844449,0.143233,0.680638,Adam,0.0010
3,0.052520,0.849761,0.139515,0.665578,Nadam,0.0010
5,0.062011,0.819389,0.136630,0.652294,RMSprop,0.0010
8,0.128877,0.603348,0.149714,0.567691,RMSprop,0.0001
7,0.138077,0.576162,0.153528,0.530675,Adam,0.0001
6,0.140273,0.570369,0.157246,0.529732,Nadam,0.0001
